In [2]:
from selenium import webdriver


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import csv

def scrape_year(year):
    driver = webdriver.Chrome()
    wait   = WebDriverWait(driver, 10)
    driver.get("https://new.giitaayan.com")

    wait.until(EC.element_to_be_clickable((
        By.XPATH, '//*[@id="root"]/div[1]/main/div/div[3]/button'))).click()

    search_box = wait.until(EC.presence_of_element_located((
        By.CSS_SELECTOR, "input[placeholder='Type words from song title']")))
    search_box.clear()
    search_box.send_keys(f"year:{year}")

    driver.find_element(
        By.XPATH, '//*[@id="root"]/div[1]/main/div/div[2]/div/form/button[2]'
    ).click()

    def harvest_current_page():
        rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
        page = []
        for r in rows:
            tds = r.find_elements(By.TAG_NAME, "td")
            if len(tds) >= 6:
                page.append([td.text.strip() for td in tds[:6]])
        return page

    all_rows = []
    while True:
        wait.until(EC.presence_of_all_elements_located((
            By.CSS_SELECTOR, "table tbody tr")))
        all_rows.extend(harvest_current_page())

        try:
            next_btn = driver.find_element(
                By.CSS_SELECTOR, "button[aria-label='Next page']")
            if not next_btn.is_enabled():       # disabled → last page
                break
            next_btn.click()
            wait.until(EC.staleness_of(next_btn))   # wait for refresh
        except (NoSuchElementException, TimeoutException):
            break                                 # arrow vanished

    driver.quit()

    with open(f"songs.csv", "a", newline='', encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerows(all_rows)




In [ ]:
for year in range(193,201):
    scrape_year(year)